In [3]:
from pytrends.request import TrendReq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import Image

# International Space Station Current Location

The ISS programme is a joint project between five participating space agencies: NASA (United States), Roscosmos (Russia), JAXA (Japan), ESA (Europe), and CSA (Canada). The ownership and use of the space station is established by intergovernmental treaties and agreements.

The ISS serves as a microgravity and space environment research laboratory in which crew members conduct experiments in biology, physics, astronomy, and other fields.

# Who are the astronauts on board right now?



In [10]:
import requests
import json
r = requests.get(url='http://api.open-notify.org/astros.json')
#Storing response
with open('astros.json', 'w') as outfile:
    json.dump(r.json(), outfile)

In [11]:
#Loading response
astros_df = pd.read_json('C:/Users/aabba/Desktop/workspace/DataScience/international-space-station/astros.json')
astros_df.head()

,message,number,people
0,success,5,"{'craft': 'ISS', 'name': 'Chris Cassidy'}"
1,success,5,"{'craft': 'ISS', 'name': 'Anatoly Ivanishin'}"
2,success,5,"{'craft': 'ISS', 'name': 'Ivan Vagner'}"
3,success,5,"{'craft': 'ISS', 'name': 'Doug Hurley'}"
4,success,5,"{'craft': 'ISS', 'name': 'Bob Behnken'}"


So there are 5 astronauts at the moment in the ISS.

# Where is the International Space Station right now ?

In [18]:
r = requests.get(url='http://api.open-notify.org/iss-now.json')
#Storing response
with open('iss-now.json', 'w') as outfile:
    json.dump(r.json(), outfile)
# Making a copy
location = r.json()

In [19]:
#Loading response
location_df = pd.read_json('C:/Users/aabba/Desktop/workspace/DataScience/international-space-station/iss-now.json')
location_df.head()

,message,iss_position,timestamp
latitude,success,40.5454,2020-06-29 02:10:57
longitude,success,21.8296,2020-06-29 02:10:57


## Let's plot the ISS current location

In [17]:
from mpl_toolkits.basemap import Basemap

# Set the dimension of the figure
plt.figure(figsize=(16, 8))

# Make the background map
m=Basemap(llcrnrlon=-180, llcrnrlat=-65,urcrnrlon=180,urcrnrlat=80)
m.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
m.fillcontinents(color='grey', alpha=0.3)
m.drawcoastlines(linewidth=0.1, color="white")


m.scatter(float(location['iss_position']['longitude']), 
          float(location['iss_position']['latitude']), 
          s=500, alpha=0.4,color='blue')

 
plt.title('International Space Station Location' , fontsize=30) 

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'